In [3]:
import pandas as pd
import numpy as np
xls = pd.read_excel(r'D:\LLM_Projects\document_portal\notebook\data\excel_original.xlsx', sheet_name=None)
#print(xls.items())


all_text = []
for sheet_name, df in xls.items():
    #remove the first row always as it is a header row
    #df = df.iloc[1:]
    df = df.replace({np.nan: None})
    all_text.append(df.to_string(index=False))
    print(f"Sheet: {sheet_name}")
print("\n".join(all_text))

all_text = "\n".join(all_text)


    

Sheet: Sheet1
Date Requested       Time Account Name Renewal New Bus Upgrade  CDB Account Number Fund Type HCR/DCR Count HCR/DCR Debit Card Y/N HCR Fund Type HRA  Count HRA Debit Card Y/N HRA Daily total counts Date Opened Date Completed Comments Listgen issue INC
     1/17/2025 11.42 a.m       Excella                 Renewal             3344821              None          None               None           HRA         66                Yes                 No   1/17/2025           None     None              None


In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional, Union
from datetime import datetime

class ExcelRow(BaseModel):
    date_requested: Union[str, datetime] = Field(..., alias="Date Requested")
    time: Optional[str] = Field(None, alias="Time")
    account_name: Optional[str] = Field(None, alias="Account Name")
    renewal_new: Optional[str] = Field(None, alias="Renewal New Bus Upgrade")
    cdb_account_number: Optional[ int] = Field(None, alias="CDB Account Number")
    fund_type_hcr_dcr: Optional[str] = Field(None, alias="Fund Type HCR/DCR")
    count_hcr_dcr: Optional[Union[str, int]] = Field(None, alias="Count HCR/DCR")
    debit_card_hcr: Optional[str] = Field(None, alias="Debit Card Y/N HCR")
    fund_type_hra: Optional[str] = Field(None, alias="Fund Type HRA")
    count_hra: Optional[Union[str, int]] = Field(None, alias="Count HRA")
    debit_card_hra: Optional[str] = Field(None, alias="Debit Card Y/N HRA")
    daily_total_counts: Optional[Union[str, int]] = Field(None, alias="Daily total counts")
    date_opened: Optional[Union[str, datetime]] = Field(None, alias="Date Opened")
    date_completed: Optional[Union[str, datetime]] = Field(None, alias="Date Completed")
    comments: Optional[str] = Field(None, alias="Comments")
    listgen_issue_inc: Optional[str] = Field(None, alias="Listgen issue INC")

class SummaryResponse1(BaseModel):
    sheet_name: str = Field(...,description="filename")
    summary: List[ExcelRow] = Field(..., alias="Summary")

In [ ]:
import pandas as pd
from pathlib import Path
#validation error
from pydantic import ValidationError
def parse_excel_to_models(excel_path: Path, sheet_name: Optional[str] = None) -> list[ExcelRow]:
    df = pd.read_excel(excel_path, sheet_name=sheet_name or 0)
    df = df.replace({np.nan: None})  # Replace NaN with None for Pydantic compatibility

    models = []
    for idx, row in df.iterrows():
        try:
            
            model = ExcelRow.model_validate(row.to_dict())
            models.append(model)
        except ValidationError as e:
            print(f"Row {idx} failed validation:\n{e}\n")

    return models

print("Parsing Excel file to Pydantic models...")
excel_path = Path(r'D:\LLM_Projects\document_portal\notebook\data\excel_original.xlsx')
models = parse_excel_to_models(excel_path)
#print(f"Parsed {len(models)} rows into Pydantic models.")
# Display the first few models for verification
#for model in models[:5]:
    #print(model.model_dump_json(indent=2, exclude_none=True))

#models to str
all_text = "\n".join([model.model_dump_json(indent=2, exclude_none=True) for model in models])
#add sheet name to the start
all_text = f"Sheet: {excel_path.stem or 'default'}\n" + all_text
print(all_text)
# excel of copy of the original file
output_excel_path = Path(r'D:\LLM_Projects\document_portal\notebook\data\excel_copy.xlsx')
copy_models = parse_excel_to_models(output_excel_path)

all_text_copy = "\n".join([model.model_dump_json(indent=2, exclude_none=True) for model in copy_models])
#add filename to the start
sheet_name = output_excel_path.stem  # Use the file name without extension as sheet name

all_text_copy = f"Sheet: {sheet_name or 'default'}\n" + all_text_copy
print(all_text_copy)




Parsing Excel file to Pydantic models...
Sheet: excel_original
{
  "date_requested": "1/17/2025",
  "time": "11.42 a.m ",
  "account_name": "Excella",
  "renewal_new": "Renewal",
  "cdb_account_number": 3344821,
  "fund_type_hra": "HRA",
  "count_hra": 66,
  "debit_card_hra": "Yes",
  "daily_total_counts": "No",
  "date_opened": "1/17/2025"
}
Sheet: excel_copy
{
  "date_requested": "1/17/2025",
  "time": "11.42 a.m ",
  "account_name": "Excella",
  "renewal_new": "Renewal",
  "cdb_account_number": 3344821,
  "fund_type_hra": "HRA",
  "count_hra": 66,
  "debit_card_hra": "Yes",
  "daily_total_counts": "No",
  "date_opened": "1/17/2025"
}
{
  "date_requested": "1/17/2025",
  "time": "11.45 a.m ",
  "account_name": "jpmc",
  "renewal_new": "new",
  "cdb_account_number": 334876,
  "fund_type_hcr_dcr": "HCR",
  "count_hcr_dcr": 79,
  "debit_card_hcr": "Y",
  "date_opened": "1/17/2025"
}
{
  "date_requested": "1/17/2025",
  "time": "11.45 a.m ",
  "account_name": "Atlanta",
  "renewal_new": 

In [ ]:
import sys
from dotenv import load_dotenv
import pandas as pd
from logger.custom_logger import CustomLogger
from exception.custom_exception import DocumentPortalException
from model.models import *
from prompt.prompt_library import PROMPT_REGISTRY
from utils.model_loader import ModelLoader
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

class DocumentComparatorLLM:
    def __init__(self):
        load_dotenv()
        self.log = CustomLogger().get_logger(__name__)
        self.loader = ModelLoader()
        self.llm = self.loader.load_llm()
        self.parser = JsonOutputParser(pydantic_object=SummaryResponse1)
        self.fixing_parser = OutputFixingParser.from_llm(parser=self.parser, llm=self.llm)
        self.prompt = ChatPromptTemplate.from_template("""
                            You will be provided with content from two excels. Your tasks are as follows:

                            1. Compare the content in two excels
                            2. Identify the difference in each row and note down the row number
                            3. The output you provide must be row  wise comparison content 
                            4. If any row do not have any change, dont mention that row 

                            Input documents:

                            {combined_docs}

                            Your response should follow this format:

                            {format_instruction}
                            """)
        self.chain = self.prompt | self.llm | self.parser
        self.log.info("DocumentComparatorLLM initialized with model and parser.")

    def compare_documents(self,combined_docs: str) -> pd.DataFrame:
        """
        Compares two documents and returns a structured comparison.
        """
        try:
            inputs = {
                "combined_docs": combined_docs,
                "format_instruction": self.parser.get_format_instructions()
            }
            self.log.info("Starting document comparison", inputs=inputs)
            response = self.chain.invoke(inputs)
            print(f"response from llm + {response}")
            self.log.info("Document comparison completed", response=response)
            return self._format_response(response)

            
        except Exception as e:
            self.log.error(f"Error in compare_documents: {e}")
            raise DocumentPortalException("An error occurred while comparing documents.",sys)
    
    def _format_response(self,response_parsed: list[dict]) -> pd.DataFrame: #type: ignore
        """
        Formats the response from the LLM into a structured format.
        """
        try:
            df = pd.DataFrame(response_parsed)
            self.log.info("Response formatted into DataFrame", dataframe=df)
            return df
        except Exception as e:
            self.log.error("Error formatting response into DataFrame", error=str(e))
            raise DocumentPortalException("Error formatting response",sys)

In [11]:
compare = DocumentComparatorLLM()
df=compare.compare_documents(all_text+"\n\n" + all_text_copy)
# Dataframe to json
print(df.to_json())


{"available_keys": ["GOOGLE_API_KEY", "GROQ_API_KEY"], "timestamp": "2025-08-08T04:11:17.267221Z", "level": "info", "event": "Environment variables validated"}
{"config_keys": ["faiss_db", "embedding_model", "retriever", "llm"], "timestamp": "2025-08-08T04:11:17.272222Z", "level": "info", "event": "Configuration loaded successfully"}
{"timestamp": "2025-08-08T04:11:17.274226Z", "level": "info", "event": "Loading LLM..."}
{"provider": "groq", "model": "deepseek-r1-distill-llama-70b", "temperature": 0, "max_tokens": 2048, "timestamp": "2025-08-08T04:11:17.277220Z", "level": "info", "event": "Loading LLM"}
{"timestamp": "2025-08-08T04:11:17.312793Z", "level": "info", "event": "DocumentComparatorLLM initialized with model and parser."}
{"inputs": {"combined_docs": "Sheet: excel_original\n{\n  \"date_requested\": \"1/17/2025\",\n  \"time\": \"11.42 a.m \",\n  \"account_name\": \"Excella\",\n  \"renewal_new\": \"Renewal\",\n  \"cdb_account_number\": 3344821,\n  \"fund_type_hra\": \"HRA\",\n 

response from llm + {'sheet_name': 'excel_copy', 'Summary': [{'Date Requested': '1/17/2025', 'Time': '11.45 a.m ', 'Account Name': 'jpmc', 'Renewal New Bus Upgrade': 'new', 'CDB Account Number': 334876, 'Fund Type HCR/DCR': 'HCR', 'Count HCR/DCR': 79, 'Debit Card Y/N HCR': 'Y', 'Date Opened': '1/17/2025'}, {'Date Requested': '1/17/2025', 'Time': '11.45 a.m ', 'Account Name': 'Atlanta', 'Renewal New Bus Upgrade': 'new', 'CDB Account Number': 45895256, 'Fund Type HCR/DCR': 'HCR', 'Count HCR/DCR': 189, 'Debit Card Y/N HCR': 'Y', 'Date Opened': '1/17/2025'}, {'Date Requested': '1/17/2025', 'Time': '11.45 a.m ', 'Account Name': 'Jockey', 'Renewal New Bus Upgrade': 'Renewal', 'CDB Account Number': 478956, 'Fund Type HCR/DCR': 'HCR', 'Count HCR/DCR': 79, 'Debit Card Y/N HCR': 'Y', 'Date Opened': '1/17/2025'}, {'Date Requested': '1/17/2025', 'Time': '11.45 a.m ', 'Account Name': 'cigna', 'Renewal New Bus Upgrade': 'new', 'CDB Account Number': 478532, 'Fund Type HRA': 'HRA', 'Count HRA': 178, '

{"response": {"sheet_name": "excel_copy", "Summary": [{"Date Requested": "1/17/2025", "Time": "11.45 a.m ", "Account Name": "jpmc", "Renewal New Bus Upgrade": "new", "CDB Account Number": 334876, "Fund Type HCR/DCR": "HCR", "Count HCR/DCR": 79, "Debit Card Y/N HCR": "Y", "Date Opened": "1/17/2025"}, {"Date Requested": "1/17/2025", "Time": "11.45 a.m ", "Account Name": "Atlanta", "Renewal New Bus Upgrade": "new", "CDB Account Number": 45895256, "Fund Type HCR/DCR": "HCR", "Count HCR/DCR": 189, "Debit Card Y/N HCR": "Y", "Date Opened": "1/17/2025"}, {"Date Requested": "1/17/2025", "Time": "11.45 a.m ", "Account Name": "Jockey", "Renewal New Bus Upgrade": "Renewal", "CDB Account Number": 478956, "Fund Type HCR/DCR": "HCR", "Count HCR/DCR": 79, "Debit Card Y/N HCR": "Y", "Date Opened": "1/17/2025"}, {"Date Requested": "1/17/2025", "Time": "11.45 a.m ", "Account Name": "cigna", "Renewal New Bus Upgrade": "new", "CDB Account Number": 478532, "Fund Type HRA": "HRA", "Count HRA": 178, "Debit C

{"sheet_name":{"0":"excel_copy","1":"excel_copy","2":"excel_copy","3":"excel_copy"},"Summary":{"0":{"Date Requested":"1\/17\/2025","Time":"11.45 a.m ","Account Name":"jpmc","Renewal New Bus Upgrade":"new","CDB Account Number":334876,"Fund Type HCR\/DCR":"HCR","Count HCR\/DCR":79,"Debit Card Y\/N HCR":"Y","Date Opened":"1\/17\/2025"},"1":{"Date Requested":"1\/17\/2025","Time":"11.45 a.m ","Account Name":"Atlanta","Renewal New Bus Upgrade":"new","CDB Account Number":45895256,"Fund Type HCR\/DCR":"HCR","Count HCR\/DCR":189,"Debit Card Y\/N HCR":"Y","Date Opened":"1\/17\/2025"},"2":{"Date Requested":"1\/17\/2025","Time":"11.45 a.m ","Account Name":"Jockey","Renewal New Bus Upgrade":"Renewal","CDB Account Number":478956,"Fund Type HCR\/DCR":"HCR","Count HCR\/DCR":79,"Debit Card Y\/N HCR":"Y","Date Opened":"1\/17\/2025"},"3":{"Date Requested":"1\/17\/2025","Time":"11.45 a.m ","Account Name":"cigna","Renewal New Bus Upgrade":"new","CDB Account Number":478532,"Fund Type HRA":"HRA","Count HRA":